# Lab 2: Grayscale Colorizer

In [1]:
import torch, torchvision
from torch.utils.data import random_split, DataLoader
from torchvision import datasets, transforms


transform = transforms.Compose([
    transforms.ToTensor(),                # [0,1]
    # optional: transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))  # [-1,1]
])

full_train = datasets.CIFAR10(root="./data", train=True, download=True, transform=transform)
test_set   = datasets.CIFAR10(root="./data", train=False, download=True, transform=transform)

# Combine train+test then re-split 80/10/10
from torch.utils.data import ConcatDataset
all_data = ConcatDataset([full_train, test_set])
n = len(all_data)
n_train = int(0.8 * n)
n_val   = int(0.1 * n)
n_test  = n - n_train - n_val

train_set, val_set, test_set = random_split(all_data, [n_train, n_val, n_test])

train_loader = DataLoader(train_set, batch_size=128, shuffle=True)
val_loader   = DataLoader(val_set, batch_size=128, shuffle=False)
test_loader  = DataLoader(test_set, batch_size=128, shuffle=False)


/Users/gustavokoglin/miniforge3/envs/cae-env/lib/python3.10/site-packages/torchvision/io/image.py:14: UserWarning: Failed to load image Python extension: 'dlopen(/Users/gustavokoglin/miniforge3/envs/cae-env/lib/python3.10/site-packages/torchvision/image.so, 0x0006): Library not loaded: @rpath/libjpeg.9.dylib
  Referenced from: <0B7EB158-53DC-3403-8A49-22178CAB4612> /Users/gustavokoglin/miniforge3/envs/cae-env/lib/python3.10/site-packages/torchvision/image.so
  Reason: tried: '/Users/gustavokoglin/miniforge3/envs/cae-env/lib/python3.10/site-packages/torchvision/../../../libjpeg.9.dylib' (no such file), '/Users/gustavokoglin/miniforge3/envs/cae-env/lib/python3.10/site-packages/torchvision/../../../libjpeg.9.dylib' (no such file), '/Users/gustavokoglin/miniforge3/envs/cae-env/lib/python3.10/lib-dynload/../../libjpeg.9.dylib' (no such file), '/Users/gustavokoglin/miniforge3/envs/cae-env/bin/../lib/libjpeg.9.dylib' (no such file)'If you don't plan on using image functionality from `torchvis

100.0%


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [6]:
import torch.nn as nn

class ConvolutionAutoEncoder(nn.Module):
    def __init__(self):
        super().__init__()

    
        #Auto encoder
        self.encoder = nn.Sequential(
                nn.Conv2d(in_channels = 3, out_channels = 8, kernel_size = 3, stride=1, padding="same"),
                nn.ReLU(),
                nn.MaxPool2d(kernel_size = 2, stride=2),
                nn.Conv2d(in_channels = 8, out_channels = 12, kernel_size = 3, stride=1, padding="same"),
                nn.ReLU(),
                nn.MaxPool2d(2, stride=2),
                nn.Conv2d(in_channels = 12, out_channels = 16, kernel_size = 3, stride=1, padding="same"),
                nn.ReLU()
            )
        self.decoder = nn.Sequential(
            nn.Upsample(scale_factor=2, mode="nearest"),
            nn.Conv2d(in_channels = 16, out_channels = 12, kernel_size = 3, stride=1, padding="same"),
            nn.ReLU(),
            nn.Upsample(scale_factor=2, mode="nearest"),
            nn.Conv2d(in_channels = 12, out_channels = 3, kernel_size = 3, stride=1, padding="same"),
            nn.ReLU()
            
        )

    def forward(self, x):
        h = self.encoder(x)
        out = self.decoder(h)
        return out, h



    

NameError: name 'self' is not defined